In [1]:
!pip3 install -U langgraph
!pip3 install -U langchain
!pip3 install grandalf
!pip3 install qianfan

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.2.32
    Uninstalling langgraph-0.2.32:
      Successfully uninstalled langgraph-0.2.32
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 1.0/1.0 MB 23.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.7
    Uninstalling langchain-core-0.3.7:
      Successfully uninstalled langchain-core-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.1
    Uninstalling langchain-0.3.1:
      Successfully uninstalled langchain-0.3.1
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [81]:

# 导入将使用模块、类和函数
from typing import TypedDict, Annotated, Union, Optional, Type, List, Annotated, Sequence
 
import requests
from pydantic import BaseModel, Field
from langchain.tools import BaseTool, tool
from langchain_core.agents import AgentAction
from langchain_core.agents import AgentFinish
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor
from langchain.chat_models import QianfanChatEndpoint
import qianfan
from langgraph.graph.message import add_messages
from functools import partial
from qianfan.extensions.langchain.agents import QianfanSingleActionAgent
from langchain.agents import AgentExecutor

import os

# 请填入您的千帆AK和SK
os.environ["QIANFAN_AK"] = "Ltg"
os.environ["QIANFAN_SK"] = "Z8ooM"

# 填写使用的模型
llm = QianfanChatEndpoint(model='ERNIE-3.5-8K')

In [82]:
#核心职责和行为准则
system_customer_support_agent='''
你是平台的客服路由工具，尽量不直接回话，而是通过工具完成用户咨询，你的主要职责是根据用户咨询的问题，提供对应的服务。如果客户修改地址，申请运费补偿等问题。请调用对应的工具将任务委托给合适的专业助手。你自己无法做出这些类型的改变只有专业助手被授权为用户执行这些操作，用户不知道不同的专业助手，因此不要提及他们，直接通过函数委派任务即可'''

#两个具体工具
@tool('ToOrderAddressUpdateAssistant')
def ToOrderAddressUpdateAssistant(content:str)->str:
    """将工作转交给专门的助手处理更新订单收货地址的事件"""
    print('调用了ToOrderAddressUpdateAssistant函数')
    return '触发后面的更新订单收货地址的助手'

@tool('ToOrderApplyShippingFeeAgent')
def ToOrderApplyShippingFeeAgent(content:str)->str:
    """将工作转交给专门的助手处理申请运费补偿的事件"""
    print('调用了ToOrderApplyShippingFeeAgent函数')
    return '触发后面的申请运费补偿的助手'

tools_customer_support_agent=[ToOrderApplyShippingFeeAgent,ToOrderAddressUpdateAssistant]
#创建客服智能体，需要三个参数，工具+聊天模型+智能体prompt
qianfan_customer_support_agent=QianfanSingleActionAgent.from_system_prompt(tools_customer_support_agent,llm,system_prompt=SystemMessage(system_customer_support_agent))
#启动该智能体
executor_customer_support_agent=AgentExecutor(agent=qianfan_customer_support_agent,tools=tools_customer_support_agent,verbose=True)

In [83]:
#构建地址更新智能体
system_addr_update_agent="""你是一个引导客户进行修改地址的助手，当客户修改地址信息的时候，主助手将委托你进行工作，根据客户提供到的信息按照以下步骤引导客户修改地址的操作。
1.如果客户没有提供订单信息，请先调用ask_for_order引导客户提供订单信息
2.如果客户提供了订单信息，请调用ask_for_address引导客户提供新的地址信息
3.如果客户提供了新的地址信息，请调用update_address更新地址

请记住，如果客户终止了意图，请调用intent_interrupt工具退出
一下是一些您应该intent_interrupt的示例：
-算了，我不想改了
-等等，我在想一下
-转人工
-再见
-我不想修改地址了
请记住，如果客户改变了意图，请调用CompleteOrEscalate交给主助手，不要创建无用的工具或函数
以下是一些您应该CompleteOrEscalate的示例：
-帮我申请运费
-帮我取消订单"""

@tool("ask_for_order")
def ask_for_order(content:str)->str:
    """向用户索要订单信息"""
    print('调用ask_for_order')
    return '请提供订单信息'

@tool("ask_for_address")
def ask_for_address(content:str)->str:
    """向用户索要地址信息"""
    print('调用ask_for_address')
    return '请提供新的地址信息'

@tool("update_address")
def update_address(content: str) -> str:
    """更新新地址"""
    print('调用update_address')
    # 调用API更新地址
    return '已完成地址更新，感谢使用'

@tool("intent_interrupt")
def intent_interrupt(content:str)->str:
    """用户取消，退出"""
    print('调用intent_interrupt')
    return '用户取消，退出'

@tool("CompleteOrEscalate")
def CompleteOrEscalate(content:str)->str:
    """A tool to mark the current task as completed and/or to escalate control of the dialog to the main assistant,
    who can re-route the dialog based on the user's needs."""
    print('调用CompleteOrEscalate')
    return '任务完成交给上游智能体'

tools_addr_update_agent=[ask_for_order,ask_for_address,update_address,intent_interrupt,CompleteOrEscalate]

qianfan_addr_update_agent=QianfanSingleActionAgent.from_system_prompt(tools_addr_update_agent,llm,system_prompt=SystemMessage(system_addr_update_agent))

executor_addr_update_agent=AgentExecutor(agent=qianfan_addr_update_agent,tools=tools_addr_update_agent,verbose=True)


In [89]:
system_apply_shipping_fee_agent = '''你是一个专门处理'运费险(也叫退换货包邮、退换运费无忧)/运费补偿'事项的助手，当用户咨询或者申请'运费险/运费补偿'的时候，主助手会将任务委托给你。
你要做的事情是返回一个工具调用，如果没有合适的工具，请将任务交还给主助手。
按以下步骤引导用户:
1. 如果用户咨询商品运费险，引导用户提供商品信息; 如果用户咨询订单运费险，引导用户提供订单信息; 如果用户申请运费险，引导用户提供订单信息; 如果用户咨询运费险(未指定是商品运费险还是订单运费险),引导用户提供商品或者订单信息。
2. 如果用户选择商品，查询商品是否支持运费险
3. 如果用户选择订单，查询订单对应的售后单列表
4. 如果没有售后单，查询订单是否超售后期: 如果超售后期，告知用户订单超售后期; 如果没有超售后期，查询订单是否支持运费险
5. 如果有多个售后单，调用工具引导用户选择售后单; 如果只有一个售后单，调用工具选择这个售后单
6. 如果用户选择的售后单不是退换货类型，调用工具告知用户售后单非退换货类型
7. 如果用户选择的售后单是退换货类型，显示运费险状态
'''


@tool("ask_for_good")
def ask_for_good() -> str:
    """向用户索要商品信息。"""
    print('调用ask_for_good')
    return '商品信息为食品'

# @tool("ask_for_order")
# def ask_for_order() -> str:
#     """向用户索要订单信息"""
#     print('调用ask_for_order')
#     return '订单号为123456789'

@tool("query_good_support")
def query_good_support() -> str:
    """查询商品是否支持运费险"""
    print('调用query_good_support')
    return '支持运费险'

@tool("query_aftersales_by_order")
def query_aftersales_by_order() -> str:
    """查询订单对应的售后单列表"""
    print('调用query_aftersales_by_order')
    return '售后单为aaaa和bbbb'

# functions = [convert_to_openai_function(t) for t in tools]



# 定义运费险助手节点
def apply_shipping_fee_agent(state):
    print('主节点正在思考，准备调用工具')
    # print(state['messages'])
    tools = [ask_for_good, query_good_support, query_aftersales_by_order]
    llm_with_tools = llm.bind_tools(tools)
    resp = llm_with_tools.invoke([SystemMessage(system)] + state['messages'])
    if len(resp.tool_calls) > 0:
        function_name = resp.tool_calls[0]['name']
        print('主节点思考完成，准备调用：' + function_name)
        return {'messages': state['messages'] + [AIMessage('调用' + function_name)], 'tool': function_name}
    else:
        return {'messages': state['messages'], 'tool': ''}


In [90]:
def ask_for_good_node(state):
    result = '商品信息为食品'
    print('调用ask_for_good_node函数，拿到结果：' + result)
    return {'messages': state['messages'] + [result], 'tool': ''}

# def ask_for_order_node(state):
#     result = '订单号为123456789'
#     print('调用ask_for_order_node函数，拿到结果：' + result)
#     return {'messages': state['messages'] + [result], 'tool': ''}

def query_good_support_node(state):
    result =  '支持运费险'
    print('调用query_good_support_node函数，拿到结果：' + result)
    return {'messages': state['messages'] + [result], 'tool': ''}

def query_aftersales_by_order_node(state) :
    result =  '售后单为aaaa和bbbb'
    print('调用query_aftersales_by_order_node函数，拿到结果：' + result)
    return {'messages': state['messages'] + [result], 'tool': ''}

In [91]:
#定义图状态
class State(TypedDict):
    tool: str
    messages: Annotated[list,add_messages]
    
graph_builder=StateGraph(State)

def customer_support_agent_rec_act(state):
    result_customer_support_agent=executor_addr_update_agent.invoke(state['messages'])
    return {'messages':[]}

def addr_customer_support_agent_rec_act(state):
    result_addr_update_agent=executor_addr_update_agent.invoke(state['messages'])
    return {'messages': [AIMessage(content=result_addr_update_agent['output'])]}

#添加节点
graph_builder.add_node("客服智能体",customer_support_agent_rec_act)
graph_builder.add_node("更新地址的助手",addr_customer_support_agent_rec_act)
graph_builder.add_node("运费险助手节点", apply_shipping_fee_agent)
graph_builder.add_node("索要商品信息", ask_for_good_node)
# graph_builder.add_node("索要订单信息", ask_for_order_node)
graph_builder.add_node("查询商品是否支持运费险", query_good_support_node)
graph_builder.add_node("查询订单对应的售后单列表", query_aftersales_by_order_node)


def decide_next_node(state):
    tool = state['tool']
    print('根据工具进行选择下一个节点，当前工具tool为：' + tool)
    if tool == 'ask_for_good':
        return '索要商品信息'
    # elif tool == 'ask_for_order':
    #     return '索要订单信息'
    elif tool == 'query_good_support':
        return '查询商品是否支持运费险'
    elif tool == 'query_aftersales_by_order':
        return '查询订单对应的售后单列表'
    else:
        return 'end'

# 添加边
graph_builder.add_conditional_edges(
    "运费险助手节点",
    decide_next_node,
    {
        '索要商品信息': '索要商品信息',
        #'索要订单信息': '索要订单信息',
        '查询商品是否支持运费险': '查询商品是否支持运费险',
        '查询订单对应的售后单列表': '查询订单对应的售后单列表',
        'end': END
    }
)

graph_builder.add_edge("客服智能体","更新地址的助手")
graph_builder.add_edge("客服智能体","运费险助手节点")
graph_builder.add_edge('索要商品信息', '运费险助手节点')
#graph_builder.add_edge('索要订单信息', '运费险助手节点')
graph_builder.add_edge('查询商品是否支持运费险', '运费险助手节点')
graph_builder.add_edge('查询订单对应的售后单列表', '运费险助手节点')

#定义主入口
graph_builder.set_entry_point("客服智能体")

#编译图
app=graph_builder.compile()

#打印图结构
app.get_graph().print_ascii()


                                  +-----------+                                  
                                  | __start__ |                                  
                                  +-----------+                                  
                                        *                                        
                                        *                                        
                                        *                                        
                                    +-------+                                    
                                    | 客服智能体 |                                    
                                    +-------+                                    
                                   **        **                                  
                                 **            **                                
                                *                *                               
                

In [92]:

## 第一轮对话：用户输入信息，想要修改地址
input_1st = "我想修改地址"

## 调用上一步创建好的应用，将用户信息传入，并获取智能体回复
result = app.invoke({
    "messages": [
        {"role": "user", "content": input_1st}
    ]
})

## 将智能体的最后回复内容放入变量，以便后续打印
output_1st = result['messages'][-1].content

## 打印对话
print("########### 第一轮对话 ##########")
print("输入：" + input_1st)
print("输出：" + output_1st)



> Entering new AgentExecutor chain...
content='' additional_kwargs={'finish_reason': 'function_call', 'request_id': 'as-7m9qy2hd13', 'object': 'chat.completion', 'search_info': [], 'function_call': {'name': 'ask_for_order', 'arguments': '{"content":"我需要获取订单信息"}'}, 'tool_calls': [{'type': 'function', 'function': {'name': 'ask_for_order', 'arguments': '{"content":"我需要获取订单信息"}'}, 'id': '790e8f24-2c0b-4267-84fd-7985364d5693'}]} response_metadata={'token_usage': {'prompt_tokens': 463, 'completion_tokens': 139, 'total_tokens': 602}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'function_call', 'id': 'as-7m9qy2hd13', 'object': 'chat.completion', 'created': 1727874369, 'result': '', 'is_truncated': False, 'need_clear_history': False, 'function_call': {'name': 'ask_for_order', 'thoughts': '用户想要修改地址，我需要调用工具来帮助他完成这个操作。首先，我需要调用[ask_for_order]来获取订单信息，然后我才能调用[ask_for_address]来获取新的地址信息，最后我才能调用[update_address]来更新地址。任务拆解：[sub-task1: 调用[ask_for_order]获取订单信息, sub-task2: 调用[ask_for_address]获取新的地址信息, s

In [93]:

## 第二轮对话：用户输入，订单号123456789
input_2nd = "我的订单是123456789"
result = app.invoke({
    "messages": [
        {"role": "user", "content": input_1st},
        {"role": "assistant", "content": output_1st},
        {"role": "user", "content": input_2nd}
    ]
})
## 拿到输出
output_2nd = result['messages'][-1].content

print("########### 第二轮对话 ##########")
print("输入：" + input_2nd)
print("输出：" + output_2nd)



> Entering new AgentExecutor chain...
content='' additional_kwargs={'finish_reason': 'function_call', 'request_id': 'as-817r8kt10e', 'object': 'chat.completion', 'search_info': [], 'function_call': {'name': 'ask_for_address', 'arguments': '{"content":"123456789"}'}, 'tool_calls': [{'type': 'function', 'function': {'name': 'ask_for_address', 'arguments': '{"content":"123456789"}'}, 'id': '564e0114-650d-4496-81cb-0d03d0215bf3'}]} response_metadata={'token_usage': {'prompt_tokens': 590, 'completion_tokens': 32, 'total_tokens': 622}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'function_call', 'id': 'as-817r8kt10e', 'object': 'chat.completion', 'created': 1727874397, 'result': '', 'is_truncated': False, 'need_clear_history': False, 'function_call': {'name': 'ask_for_address', 'thoughts': '用户提供了订单信息，现在我可以调用工具来帮助他修改地址。', 'arguments': '{"content":"123456789"}'}, 'usage': {'prompt_tokens': 590, 'completion_tokens': 32, 'total_tokens': 622}} id='run-b56793e8-04df-4263-ace1-71f5d3bf6f1f-0' 

[ERROR][2024-10-02 21:06:30.826] openapi_requestor.py:296 [t:33508]: api request req_id: as-akdxtq99c6 failed with error code: 336006, err msg: the length of messages must be an odd number, please check https://cloud.baidu.com/doc/WENXINWORKSHOP/s/tlmyncueh


主节点思考完成，准备调用：query_aftersales_by_order
根据工具进行选择下一个节点，当前工具tool为：query_aftersales_by_order
调用query_aftersales_by_order_node函数，拿到结果：售后单为aaaa和bbbb
主节点正在思考，准备调用工具


APIError: api return error, req_id: as-akdxtq99c6 code: 336006, msg: the length of messages must be an odd number

In [64]:

## 第三轮对话：用户输入，地址是上海市杨浦区
input_3rd = "算了"
result = app.invoke({
    "messages": [
        {"role": "user", "content": input_1st},
        {"role": "assistant", "content": output_1st},
        {"role": "user", "content": input_2nd},
        {"role": "assistant", "content": output_2nd},
        {"role": "user", "content": input_3rd},
    ]
})
## 拿到输出
output_3rd = result['messages'][-1].content

print("########### 第三轮对话 ##########")
print("输入：" + input_3rd)
print("输出：" + output_3rd)



> Entering new AgentExecutor chain...
content='' additional_kwargs={'finish_reason': 'function_call', 'request_id': 'as-vmka1hqy8v', 'object': 'chat.completion', 'search_info': [], 'function_call': {'name': 'intent_interrupt', 'arguments': '{"content":"算了"}'}, 'tool_calls': [{'type': 'function', 'function': {'name': 'intent_interrupt', 'arguments': '{"content":"算了"}'}, 'id': '5a21051b-7289-47f4-89cc-a5e51a806948'}]} response_metadata={'token_usage': {'prompt_tokens': 703, 'completion_tokens': 138, 'total_tokens': 841}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'function_call', 'id': 'as-vmka1hqy8v', 'object': 'chat.completion', 'created': 1727868475, 'result': '', 'is_truncated': False, 'need_clear_history': False, 'function_call': {'name': 'intent_interrupt', 'thoughts': '用户想修改地址；我需要帮助用户修改地址；根据ask_for_order工具的定义，全部参数集合为[content]；其中"required": true的必要参数集合为[content]；结合用户当前问题“算了”和整个对话历史，用户已经提供了以下参数值{content: \'算了\'}；其中已经提供对应参数值的"required": true的必要参数集合为[content]；尚未提供对应参数值的"required

In [47]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.utils.function_calling import convert_to_openai_function
from langgraph.graph import END

In [48]:

# 流程主控节点system指令，根据流程逐步执行
system = '''你是一个专门处理'运费险(也叫退换货包邮、退换运费无忧)/运费补偿'事项的助手，当用户咨询或者申请'运费险/运费补偿'的时候，主助手会将任务委托给你。
你要做的事情是返回一个工具调用，如果没有合适的工具，请将任务交还给主助手。
按以下步骤引导用户:
1. 如果用户咨询商品运费险，引导用户提供商品信息; 如果用户咨询订单运费险，引导用户提供订单信息; 如果用户申请运费险，引导用户提供订单信息; 如果用户咨询运费险(未指定是商品运费险还是订单运费险),引导用户提供商品或者订单信息。
2. 如果用户选择商品，查询商品是否支持运费险
3. 如果用户选择订单，查询订单对应的售后单列表
4. 如果没有售后单，查询订单是否超售后期: 如果超售后期，告知用户订单超售后期; 如果没有超售后期，查询订单是否支持运费险
5. 如果有多个售后单，调用工具引导用户选择售后单; 如果只有一个售后单，调用工具选择这个售后单
6. 如果用户选择的售后单不是退换货类型，调用工具告知用户售后单非退换货类型
7. 如果用户选择的售后单是退换货类型，显示运费险状态
'''

@tool("ask_for_good")
def ask_for_good() -> str:
    """向用户索要商品信息。"""
    print('调用ask_for_good函数')
    return '商品信息为食品'


@tool("ask_for_order")
def ask_for_order() -> str:
    """向用户索要订单信息"""
    print('调用ask_for_order')
    return '订单号为123456789'

@tool("query_good_support")
def query_good_support() -> str:
    """查询商品是否支持运费险"""
    print('调用query_good_support')
    return '支持运费险'

@tool("query_aftersales_by_order")
def query_aftersales_by_order() -> str:
    """查询订单对应的售后单列表"""
    print('调用query_aftersales_by_order')
    return '售后单为aaaa和bbbb'

# 定义运费险助手节点
def apply_shipping_fee_agent(state):
    print('主节点正在思考，准备调用工具')
    # print(state['messages'])
    tools = [ask_for_order, ask_for_good, query_good_support, query_aftersales_by_order]
    llm_with_tools = llm.bind_tools(tools)
    resp = llm_with_tools.invoke([SystemMessage(system)] + state['messages'])
    if len(resp.tool_calls) > 0:
        function_name = resp.tool_calls[1]['name']
        print('主节点思考完成，准备调用：' + function_name)
        return {'messages': state['messages'] + [AIMessage('调用' + function_name)], 'tool': function_name}
    else:
        return {'messages': state['messages'], 'tool': ''}

In [49]:

# 用户交互商品信息的节点
def ask_for_good_node(state):
    result = '商品信息为食品'
    print('调用ask_for_good_node函数，拿到结果：' + result)
    return {'messages': state['messages'] + [result], 'tool': ''}

# 用户交互订单信息的节点
def ask_for_order_node(state):
    result = '订单号为123456789'
    print('调用ask_for_order_node函数，拿到结果：' + result)
    return {'messages': state['messages'] + [result], 'tool': ''}

# 查询商品是否支持运费险的节点
def query_good_support_node(state):
    result =  '支持运费险'
    print('调用query_good_support_node函数，拿到结果：' + result)
    return {'messages': state['messages'] + [result], 'tool': ''}

# 查询售后单的节点
def query_aftersales_by_order_node(state) :
    result =  '售后单为aaaa和bbbb'
    print('调用query_aftersales_by_order_node函数，拿到结果：' + result)
    return {'messages': state['messages'] + [result], 'tool': ''}

In [50]:

# 定义图状态
class State(TypedDict):
    # messages have the type "list".
    # The add_messages function appends messages to the list, rather than overwriting them
    tool: str
    messages: Annotated[list, add_messages]

# 定义图对象
graph_builder = StateGraph(State)

# 添加节点
graph_builder.add_node("运费险助手节点", apply_shipping_fee_agent)
graph_builder.add_node("索要商品信息", ask_for_good_node)
graph_builder.add_node("索要订单信息", ask_for_order_node)
graph_builder.add_node("查询商品是否支持运费险", query_good_support_node)
graph_builder.add_node("查询订单对应的售后单列表", query_aftersales_by_order_node)

def decide_next_node(state):
    tool = state['tool']
    print('根据工具进行选择下一个节点，当前工具tool为：' + tool)
    if tool == 'ask_for_good':
        return '索要商品信息'
    elif tool == 'ask_for_order':
        return '索要订单信息'
    elif tool == 'query_good_support':
        return '查询商品是否支持运费险'
    elif tool == 'query_aftersales_by_order':
        return '查询订单对应的售后单列表'
    else:
        return 'end'

# 添加边
graph_builder.add_conditional_edges(
    "运费险助手节点",
    decide_next_node,
    {
        '索要商品信息': '索要商品信息',
        '索要订单信息': '索要订单信息',
        '查询商品是否支持运费险': '查询商品是否支持运费险',
        '查询订单对应的售后单列表': '查询订单对应的售后单列表',
        'end': END
    }
)
graph_builder.add_edge('索要商品信息', '运费险助手节点')
graph_builder.add_edge('索要订单信息', '运费险助手节点')
graph_builder.add_edge('查询商品是否支持运费险', '运费险助手节点')
graph_builder.add_edge('查询订单对应的售后单列表', '运费险助手节点')


# 定义主入口
graph_builder.set_entry_point("运费险助手节点")

# 编译图
app = graph_builder.compile()

## 打印图结构
app.get_graph().print_ascii()

                                              +-----------+                                            
                                              | __start__ |                                            
                                              +-----------+                                            
                                                    *                                                  
                                                    *                                                  
                                                    *                                                  
                                              +---------+                                              
                                           ***| 运费险助手节点 |**...                                         
                                   ********  *+---------+  ***.........                                
                           ********      ****         .       **

In [55]:

# 用户输入，想要修改地址
input_1st = "咨询商品运费险"

result = app.invoke({
    "messages": [{"role": "user", "content": input_1st}]
})
## 拿到输出
print(result)

主节点正在思考，准备调用工具
主节点思考完成，准备调用：ask_for_good
根据工具进行选择下一个节点，当前工具tool为：ask_for_good
调用ask_for_good_node函数，拿到结果：商品信息为食品
主节点正在思考，准备调用工具
主节点思考完成，准备调用：query_good_support
根据工具进行选择下一个节点，当前工具tool为：query_good_support
调用query_good_support_node函数，拿到结果：支持运费险
主节点正在思考，准备调用工具
根据工具进行选择下一个节点，当前工具tool为：
{'tool': '', 'messages': [HumanMessage(content='咨询商品运费险', additional_kwargs={}, response_metadata={}, id='b86b468f-9c1e-4e09-9bcc-b3e738597bff'), AIMessage(content='调用ask_for_good', additional_kwargs={}, response_metadata={}, id='35dc9018-8a69-4e7c-ab36-913e0a91121d'), HumanMessage(content='商品信息为食品', additional_kwargs={}, response_metadata={}, id='d42a2f37-76a2-4256-a2ff-8ff2528fa7ae'), AIMessage(content='调用query_good_support', additional_kwargs={}, response_metadata={}, id='9839af9b-3c8b-41bf-abaa-84fb1a8e214b'), HumanMessage(content='支持运费险', additional_kwargs={}, response_metadata={}, id='235f1bbf-aed4-44e5-9bff-46563b341d0f')]}
